In [1]:
import os
os.chdir('../')

In [2]:
import pandas as pd
from pathlib import Path
from PIL import Image
import json
import numpy as np
import subprocess

In [3]:
root_path = Path('./data/')

In [4]:
style_df = pd.read_csv(root_path / 'metadata' / 'style.csv')
style_df = style_df[~style_df['File_name'].isna()]
style_df = style_df.sort_values('File_name').reset_index(drop=True)

content_df = pd.read_csv(root_path / 'metadata' / 'content.csv')
content_df = content_df[~content_df['File_name'].isna()]
content_df = content_df.sort_values('File_name').reset_index(drop=True)

In [5]:
n_random=200
content_indices = np.random.randint(0, content_df.shape[0], n_random)
style_indices = np.random.randint(0, style_df.shape[0], n_random)

In [7]:
content_path = root_path / 'content-images'
style_path = root_path / 'style-images'

metadata_path = root_path / 'output' / 'metadata'
metadata_path.mkdir(exist_ok=True, parents=True)

In [ ]:
for i, (content_id, style_id) in enumerate(zip(content_indices, style_indices)):
    content = content_df.iloc[content_id]
    style = style_df.iloc[style_id]
    
    content_img_path = content_path / content['File_name']
    style_img_path = content_path / style['File_name']
    output_name = f'random_{i}'
    
    metadata = {
                "description": "The most iconic pieces of art, reimagined by AI.",
                "image": "TBD",
                "name" : f"{content['Title']} X {style['Title']}",
                "animation_url": "TBD",
                "attributes": [
                    {
                        "trait_type": "Content",
                        "value": content['Title']
                    },
                    {
                        "trait_type": "Content Author",
                        "value": content['Author']
                    },
                    {
                        "trait_type": "Style",
                        "value": style['Title']
                    },
                    {
                        "trait_type": "Style Author",
                        "value": style['Author']
                    },
                    {
                        "trait_type": "Orientation",
                        "value": content['Orientation']
                    },
                    {
                        "trait_type": "File Name",
                        "value": output_name
                    },
                    {
                        "trait_type": "Style weight",
                        "value": 6e4
                    }
                ]
            }
    
    print(f"Processing content image: {content['File_name']}")
    print(f"Processing style: {style['File_name']}")
    print(f"Processing output name: {output_name}")
    print(json.dumps(metadata, indent=4))

    !python neural_style_transfer.py --content_img_name {content['File_name']} --style_img_name {style['File_name']} --output_img_name {output_name} --output_path "random"

    file_metadata_path = metadata_path / (output_name + '.json')
    with open(file_metadata_path, 'w') as f:
        json.dump(metadata, f)
    
    print()

Processing content image: luncheon_on_the_grass.jpg
Processing style: dance_hall_bellevue.jpg
Processing output name: random_0
{
    "description": "The most iconic pieces of art, reimagined by AI.",
    "image": "TBD",
    "name": "Luncheon on the Grass X Dance Hall Bellevue",
    "animation_url": "TBD",
    "attributes": [
        {
            "trait_type": "Content",
            "value": "Luncheon on the Grass"
        },
        {
            "trait_type": "Content Author",
            "value": "Edouard Manet"
        },
        {
            "trait_type": "Style",
            "value": "Dance Hall Bellevue"
        },
        {
            "trait_type": "Style Author",
            "value": "Ludwig Kirchner"
        },
        {
            "trait_type": "Orientation",
            "value": "Landscape"
        },
        {
            "trait_type": "File Name",
            "value": "random_0"
        },
        {
            "trait_type": "Style weight",
            "value": 60000.0